## Compaines' historical Fama French 3-Factors and CAPM
*Melissa Putur,
Aparna Raman,
Luke Towers, 
Adil Wahab,
Di Yao* 

### 1. Load the dataset

In [ ]:
#Loading packages
from google.colab import files
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
# Upload stock returns file
uploaded = files.upload()

Saving factors.csv to factors (1).csv


In [ ]:
# Upload factor returns file 
uploaded = files.upload()

Saving returns.csv to returns.csv


In [ ]:
# Convert the files to DFs
df1 = pd.read_csv('returns.csv')
df2 = pd.read_csv('factors.csv')

### 2. Data Cleaning

Rename two dataframe columns in order to merge them into one dataset.

In [ ]:
# Rename stock returns columns
df1.rename(columns = {'Names Date':'Date', 'Ticker Symbol':'Ticker','Returns':'Ret'}, inplace=True)

In [ ]:
# Rename factor returns columns 
df2.rename(columns = {'Date (SAS). Last Trading Day of the Month':'Date', 'Small-Minus-Big Return':'SMB','High-Minus-Low Return':'HML', 
                      'Risk-Free Return Rate (One Month Treasury Bill Rate)':'Rf',
                      'Excess Return on the Market':'RmMinusRf'}, inplace=True)

In [ ]:
# Merge returns(df1) and factors(df2) tables into df
df = pd.merge(df1, df2, on='Date', how='left')

Delete tickers that have less than 24 months of observations to make sure the regression has reasonable amount of data points. <br>
Updating tickers based on PERMNO column and removing rows with missing tickers.

In [ ]:
# Deleting any tickers that have < 2 years of data
dataCount = df.groupby('PERMNO').count()
dataCount2 = dataCount.filter(["Ticker","PERMNO"], axis = 1)
dataCount2.rename(columns = {'Ticker':'count'}, inplace = True)
df = pd.merge(df, dataCount2, on='PERMNO', how='left')
indexToDelete = df[df['count'] < 24].index
df.drop(indexToDelete, inplace = True)

In [ ]:
# Updating missing tickers, and removing some rows with missing tickers
df.loc[df.PERMNO == 93369, 'Ticker'] = "RCM"
df.loc[df.PERMNO == 15936, 'Ticker'] = "BGNE"
df.loc[df.PERMNO == 92102, 'Ticker'] = "SCOR"
df.loc[df.PERMNO == 86536, 'Ticker'] = "DFFN"
df.loc[df.PERMNO == 15937, 'Ticker'] = "EDIT"
df.loc[df.PERMNO == 15960, 'Ticker'] = "CRVS"

df.dropna(inplace=True)

Replace return columns' missing value with this company's average return.

In [ ]:
# Replacing NA values with mean values for the group
df['Ret'] = df['Ret'].replace('C', np.NaN)
df['Ret'] = df['Ret'].replace('B', np.NaN)

# Convert returns column to numeric 
df[['Ret']] = df[['Ret']].apply(pd.to_numeric)

In [ ]:
# Replace missing returns values with mean of the specific stock's returns
df['Ret'] = df['Ret'].fillna(df.groupby('Ticker')['Ret'].transform('mean'))

In [ ]:
# Create R Minus Rf column
df['RMinusRf'] = df['Ret'] - df['Rf']

In [ ]:
df.shape

(60641, 10)

In [ ]:
# Glimpse of merged DF
df.head()

,PERMNO,Date,Ticker,Ret,RmMinusRf,SMB,HML,Rf,count,RMinusRf
0,10026,20160129,JJSF,-0.074484,-0.0577,-0.0335,0.0208,0.0001,48,-0.074584
1,10026,20160229,JJSF,0.026023,-0.0007,0.0079,-0.0050,0.0002,48,0.025823
2,10026,20160331,JJSF,-0.019135,0.0696,0.0087,0.0116,0.0002,48,-0.019335
3,10026,20160429,JJSF,-0.066033,0.0092,0.0069,0.0326,0.0001,48,-0.066133
4,10026,20160531,JJSF,0.043212,0.0178,-0.0027,-0.0181,0.0001,48,0.043112


### 3. CAPM Beta

In [ ]:
# Create loop for CAPM betas and put into DF
ticker = np.unique(df['Ticker'])
beta = []
for i in ticker:
  subset = df.loc[df['Ticker'] == i] 
  X = subset['RmMinusRf']
  y = subset['RMinusRf']
  X = sm.add_constant(X)
  model = sm.OLS(y, X).fit()
  coef = model.params.RmMinusRf
  beta.append(coef)
print(beta)
CAPM = pd.DataFrame({'ticker':ticker, 'CAPM_Beta':beta})
CAPM.head()

[1.2493624622078516, 1.803726938683534, 0.9500440497155062, 1.0068597757254494, 1.172580298180833, 2.2769392936824917, 0.622612937977445, 0.8089445483230095, 1.1309483945190346, 1.7841724369377518, 1.2759378556916314, 0.9311490907049249, 2.760780943923268, 1.474851208642583, 2.688727911834322, 0.7055471207278148, 0.7848670250607978, 1.7192501312806037, 1.2880047002537, 1.383904658537475, 0.9576774562377661, 1.3242223634653216, 1.6565216379946768, 1.09632169248087, 1.4848430926096672, 0.9466460716630437, 3.1191253508444303, 1.8044646064145642, 0.8300202228710176, 1.1527991054252522, 1.972988815316974, 1.623293629614101, 0.24674455749124624, 0.13635181386561382, 2.1083740047575095, 1.7236731030017238, 0.056916805340787247, 0.6030117935709856, 0.7934873858739508, 0.48318050151579683, 0.6427300650565742, 2.650404239693419, 1.5780562216657992, 0.2671560624731304, 0.00612664233347772, 1.029813564875201, 2.1137626857308756, 0.8282847985389931, 0.5200448717174553, 0.5434048491046066, 0.6534190

,ticker,CAPM_Beta
0,A,1.249362
1,AAL,1.803727
2,AAON,0.950044
3,AAP,1.006860
4,AAPL,1.172580


In [ ]:
CAPM.shape

(1361, 2)

In [ ]:
# Descriptive Statistics of CAPM Model
ddf = pd.DataFrame(CAPM.describe())
CAPM_des = ddf.drop(['count', 'std'])
CAPM_des

,CAPM_Beta
mean,1.193104
min,-5.229175
25%,0.752256
50%,1.163310
75%,1.548536
max,6.277273


### 4. Fama-French 3-Factor Model

In [ ]:
# Create loop for FF coefficients and put into DF
ticker = np.unique(df['Ticker'])
beta = []
size = []
value = []
for i in ticker:
  subset = df.loc[df['Ticker'] == i] 
  X = subset[['RmMinusRf', 'SMB', 'HML']]
  y = subset['RMinusRf']
  X = sm.add_constant(X)
  model = sm.OLS(y, X).fit()
  coef1 = model.params.RmMinusRf
  beta.append(coef1)
  coef2 = model.params.SMB
  size.append(coef2)
  coef3 = model.params.HML
  value.append(coef3)
print(beta)
print(size)
print(value)
FF = pd.DataFrame({'ticker':ticker, 'FF_Beta':beta, 'FF_size':size,'FF_value':value})
FF.head()

[1.3978103196043847, 1.7988552604686967, 0.9289649934894555, 0.8759471515070184, 1.296142814191379, 1.9287490853768259, -0.03776046434243475, 0.7532755833756295, 1.096687501147176, 1.4461221338382262, 0.776887441825017, 0.9865681621726661, 2.2668336339882202, 1.7242576082349477, 2.5425892623121396, 0.759869041088193, 0.5221027145444064, 1.3070767186885308, 1.2450746756270257, 1.3167993251733887, 1.0268427943961538, 0.6421681676521628, 0.8077345104529909, 1.1505413991396098, 1.4301237804804439, 0.9754896726305167, 2.916815700045368, 1.0971959606396369, 0.7508626724598543, -0.01882207408930113, 1.7986966627959788, 1.5761285013534254, -0.32383238564396005, 0.20701398867644982, 1.8204697185976697, 1.6585570768672595, 0.20436437992025563, 0.03097991582414636, 0.772445651816709, 0.4792406415050898, 0.6625997079272343, 2.2422623648303706, 1.6575922627566089, 0.24076189257317362, 0.001807849925722842, 1.1370226462761912, 1.9973928028457522, 0.41556318797356595, 0.5179732529027778, 0.3486555539

In [ ]:
FF.shape

(1361, 4)

In [ ]:
# Descriptive Statistics of FF Model
ddf = pd.DataFrame(FF.describe())
FF_des = ddf.drop(['count', 'std'])
FF_des

,FF_Beta,FF_size,FF_value
mean,1.059012,0.513168,0.026301
min,-8.016714,-10.218794,-9.170109
25%,0.640896,-0.065438,-0.435950
50%,1.024698,0.382206,0.026731
75%,1.410660,0.975648,0.577495
max,6.698717,9.826025,7.609125


### 5. Combine FF and CAPM Models


**Interpreting the components of the two models** <br>
*Capital Asset Pricing Model (CAPM)* <br>
- Alpha(constant): difference between stock's actual return and expected CAPM return; measure of reward or penalty for holding firm-specific risk <br>
- Beta: measures systematic risk (performance of overall economy) of an investment, which refers to the sensitivity of stock to the movement of the market index benchmark <br>
High Beta = more volatile than market benchmark <br>
Low Beta = less volatile than market <br>
Based on our regression result, mean CAMP is 1.19 > 1, which indicates these stocks on average is theoretically 19% more volatile than the market. 


*Fama-French Three-Factor Model (FF)* <br>
- FF_Beta is 1.05 on average, which shows these stocks are 5% more volatile than the market.
- SMB coefficient: measures small firm risk AKA Small Minus Big companies <br>
Positive coefficient = firm has small firm risk <br>
Negative coefficient = firm has big firm risk <br>
Our result shows that these firms have on averge of 0.513 , which shows firms has small firm risks.<br>
- HML coefficient: measures the outperformance of value stocks over growth stocks AKA High Minus Low value; value stocks are firms with high book-to-market ratios while growth stocks are firms with low book-to-market ratios <br>
Positive coefficient = a value/distress firm with high sensitivity to economic performance where it will perform really well if economy is experiencing positive movements, however is expected to perform extremely poorly during economic downturns <br>
Negative coefficient = a growth stock less sensitive to changes in economic performance <br>
As the result shows, HML coeffecient is 0.026 on average, which indicates these firms perform well as economy performance well. 

In [ ]:
# Compare descriptive statistics of two models as a DF
des_stats = pd.concat([CAPM_des, FF_des], axis=1)
des_stats

,CAPM_Beta,FF_Beta,FF_size,FF_value
mean,1.193104,1.059012,0.513168,0.026301
min,-5.229175,-8.016714,-10.218794,-9.170109
25%,0.752256,0.640896,-0.065438,-0.435950
50%,1.163310,1.024698,0.382206,0.026731
75%,1.548536,1.410660,0.975648,0.577495
max,6.277273,6.698717,9.826025,7.609125


In [ ]:
# Use merge to combine CAPM and FF coefficients
df_mods = pd.merge(CAPM, FF, on='ticker', how='left')
df_mods.rename(columns={'ticker':'Ticker','FF_Beta':'FF-Beta','FF_size':'FF-Size','FF_value':'FF-Value'},inplace=True)
df_mods.head()

,Ticker,CAPM_Beta,FF-Beta,FF-Size,FF-Value
0,A,1.249362,1.397810,-0.540208,-0.310123
1,AAL,1.803727,1.798855,-0.025965,0.686844
2,AAON,0.950044,0.928965,0.078249,0.020157
3,AAP,1.006860,0.875947,0.453546,0.627367
4,AAPL,1.172580,1.296143,-0.416180,-0.776466


In [ ]:
# Get ccwd
import os
os.getcwd()

'/content'

In [ ]:
# Export df_mods into an xlsx file in cwd
df_mods.to_excel (r'/content/Assign4-Output.xlsx', index = False, header=True)